In [15]:
! pip install numpy
! pip install pandas
! pip install seaborn
! pip install scikit-learn
! pip install matplotlib
! pip install tensorflow
! pip install keras
! pip install sgp4
! pip install scipy

! pip install openpyxl



   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpy

In [16]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sgp4.api import Satrec
from scipy.spatial.transform import Rotation as R

# TLE da ISS
line1 = "1 25544U 98067A   24106.45347222  .00012190  00000+0  21377-3 0  9992"
line2 = "2 25544  51.6414  52.8843 0003046  96.6780  42.0736 15.51762384294959"
sat = Satrec.twoline2rv(line1, line2)

# Função auxiliar
def datetime_to_julian_date(dt):
    year, month, day = dt.year, dt.month, dt.day
    hour, minute, second = dt.hour, dt.minute, dt.second + dt.microsecond / 1e6
    jd = 367 * year - int((7 * (year + int((month + 9) / 12))) / 4) \
         + int((275 * month) / 9) + day + 1721013.5 \
         + (hour + minute / 60 + second / 3600) / 24
    fr = (hour + minute / 60 + second / 3600) / 24 % 1
    return jd, fr

# Parâmetros
base_time = datetime.utcnow()
intervalo_segundos = 30
samples = 200
tempos, q0, q1, q2, q3 = [], [], [], [], []
rolls, pitchs, yaws = [], [], []

# Loop de propagação
for i in range(samples):
    dt = base_time + timedelta(seconds=i * intervalo_segundos)
    jd, fr = datetime_to_julian_date(dt)
    err, position, velocity = sat.sgp4(jd, fr)

    if err != 0:
        print(f"Erro na propagação para t={dt}, código: {err}")
        continue

    r = np.array(position)
    v = np.array(velocity)

    z = -r / np.linalg.norm(r)
    y = np.cross(z, v); y /= np.linalg.norm(y)
    x = np.cross(y, z)
    rot_matrix = np.vstack([x, y, z]).T
    rotation = R.from_matrix(rot_matrix)

    quat = rotation.as_quat()   # x, y, z, w
    euler = rotation.as_euler('xyz', degrees=True)

    tempos.append(i * intervalo_segundos)
    q0.append(quat[3]); q1.append(quat[0]); q2.append(quat[1]); q3.append(quat[2])
    rolls.append(euler[0]); pitchs.append(euler[1]); yaws.append(euler[2])

# Rotulagem automática
def rotular_gimbal_lock(pitch, tolerancia=2):
    if abs(pitch - 90) < tolerancia:
        return "gimbal_lock"
    elif abs(pitch - 90) < tolerancia + 5:
        return "alerta"
    else:
        return "ok"

# DataFrame final
df = pd.DataFrame({
    'tempo': tempos,
    'q0': q0, 'q1': q1, 'q2': q2, 'q3': q3,
    'roll': rolls,
    'pitch': pitchs,
    'yaw': yaws
})
df['status'] = df['pitch'].apply(rotular_gimbal_lock)

print(df.head())
df.to_excel("dados_orientacao_IA.xlsx", index=False) # visualização


   tempo        q0        q1        q2        q3       roll      pitch  \
0      0 -0.491264  0.656470  0.021570  0.572050 -77.531516 -50.557348   
1     30 -0.490239  0.675605  0.038324  0.549322 -82.272072 -51.244618   
2     60 -0.488643  0.693957  0.055033  0.525949 -87.158319 -51.605725   
3     90 -0.486480  0.711503  0.071678  0.501959 -92.100206 -51.631097   
4    120 -0.483752  0.728222  0.088240  0.477379 -97.002082 -51.319916   

         yaw status  
0 -57.153155     ok  
1 -51.047040     ok  
2 -44.799427     ok  
3 -38.497579     ok  
4 -32.234366     ok  


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical

# 1. Carrega o dataframe salvo anteriormente
df = pd.read_excel("dados_orientacao_IA.xlsx")

# 2. Seleciona colunas de entrada e saída
features = ['pitch', 'roll', 'yaw', 'q0', 'q1', 'q2', 'q3']
target = 'status'

# 3. Codifica rótulos ("ok", "alerta", "gimbal_lock") para inteiros e one-hot
label_encoder = LabelEncoder()
df['status_encoded'] = label_encoder.fit_transform(df[target])
y = to_categorical(df['status_encoded'])  # 0=alerta, 1=gimbal_lock, 2=ok (ou vice-versa)

# 4. Normaliza os dados de entrada
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df[features])

# 5. Cria janelas temporais para LSTM (ex: 10 passos por entrada)
def criar_janelas(X, y, janela=10):
    X_out, y_out = [], []
    for i in range(janela, len(X)):
        X_out.append(X[i - janela:i])
        y_out.append(y[i])
    return np.array(X_out), np.array(y_out)

janela_temporal = 10
X_seq, y_seq = criar_janelas(X_scaled, y, janela_temporal)

# 6. Divide em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# 7. Define modelo LSTM
model = Sequential()
model.add(LSTM(64, input_shape=(janela_temporal, len(features))))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))  # 3 classes: ok, alerta, gimbal_lock

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 8. Treina o modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2)

# 9. Avalia o modelo
loss, acc = model.evaluate(X_test, y_test)
print(f"Acurácia final no conjunto de teste: {acc*100:.2f}%")

# 10. Exemplo de previsão
preds = model.predict(X_test[:5])
print("Previsões:", label_encoder.inverse_transform(np.argmax(preds, axis=1)))
